## Introduction

- Tao đang kiểm tra nhẹ làm thế nào để run một chiếc spark load tensorflow model nha mấy bé. 

- mấy bé thấy hay thì nhớ lài lái lai và sè sé se. và follow github cho tao nha: [Github lè](https://github.com/dnguyenngoc) or [Github group lè](https://github.com/apot-group)

<div class="alert alert-block alert-info">
<b>Step 1:</b> Import Libraries
</div>

In [9]:
# Spark Session, Pipeline, Functions, and Metrics
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.ml import Pipeline
import pandas as pd
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.2,' \
                                    + 'org.apache.spark:spark-streaming-kafka-0-10_2.12:3.0.2,' \
                                    + 'org.apache.kafka:kafka-clients:3.1.0,org.apache.spark:spark-core_2.12:3.0.2,' \
                                    + 'org.apache.spark:spark-streaming_2.12:3.0.2,' \
                                    + 'org.apache.spark:spark-token-provider-kafka-0-10_2.12:3.0.2,' \
                                    + 'org.apache.hadoop:hadoop-core:1.2.1,' \
                                    + 'org.tensorflow:tensorflow-lite:2.8.0' \
                                    + ' pyspark-shell'

<div class="alert alert-block alert-info">
<b>Step 2:</b> Start Spark Session
</div>

In [10]:
ss  = SparkSession.builder.master('local[1]') \
                  .appName("test") \
                  .config("spark.executor.memory", "2g") \
                  .config("spark.driver.memory", "2g") \
                  .config("spark.executor.cores", "2") \
                  .config("spark.driver.cores", "2") \
                  .getOrCreate()
ss

<div class="alert alert-block alert-info">
<b>Step 3:</b> Load tensoflow model
</div>

In [12]:
zero_df = ss.read.format("image").load('/work/data/images/')

In [14]:
zero_df.show()

+--------------------+
|               image|
+--------------------+
|{file:///work/dat...|
|{file:///work/dat...|
|{file:///work/dat...|
|{file:///work/dat...|
|{file:///work/dat...|
|{file:///work/dat...|
|{file:///work/dat...|
|{file:///work/dat...|
|{file:///work/dat...|
+--------------------+



In [ ]:
!pip3 install tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD
model = Sequential()
model.add(Dense(128, input_dim=784))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=SGD())

     |██▍                             | 37.2 MB 141 kB/s eta 0:54:11     |█▏                              | 17.3 MB 833 kB/s eta 0:09:37     |█▋                              | 25.6 MB 233 kB/s eta 0:33:42     |██▎                             | 35.8 MB 680 kB/s eta 0:11:19     |██▎                             | 35.9 MB 680 kB/s eta 0:11:19     |██▍                             | 36.4 MB 333 kB/s eta 0:23:02